In [1]:
import pandas as pd
import os 

In [2]:
# define the base path to thumb drive (input path)
base_path = '/Volumes/Untitled/MeterDataTest'

#check if base path exists
if not os.path.exists(base_path):
    print(f"Error: Path {base_path} does not exist")
    exit()

In [3]:
# list to store all dataframes
all_data = []

#list to store all dataframes with 3_phase_watt_total column for later use
save_data = []

# dataframe for all data
combined_data = pd.DataFrame()
save_data_col = pd.DataFrame()

In [ ]:
# iterate through the subfolders in the MeterDataTest folder
for subfolder in os.listdir(base_path):
    subfolder.replace(" ", "_")
    # create path for each subfolder
    folder_path = os.path.join(base_path, subfolder)

    # get the name of the meter from the subfolder name, make lowercase
    meter_name = subfolder.lower().replace(" ", "_") 

    # list of csv file paths in subfolder
    # addition with the and not is to make sure to ignore the hidden ._ files
    csv_paths = [os.path.join(folder_path, f) 
                 for f in os.listdir(folder_path) 
                 if f.endswith('.csv')
                 and not f.startswith("._")
                 and not f.startswith(".")]

    # convert each csv to a df, fix columns and add df to df list
    for csv in csv_paths:
        df = pd.read_csv(csv, encoding="utf-8")

        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
        
        if '3_phase_positive_real_energy_used' in df.columns:
            df.rename(columns={'3_phase_positive_real_energy_used': 'total_watt_hour', '3_phase_real_power':'3_phase_watt_total'}, inplace=True)

        # reorder the columns
        df = df[['datetime', 'total_watt_hour', '3_phase_watt_total']]

        df.insert(1, 'meter_name', meter_name)

        # save the dataframe with all columns to list
        save_data.append(df.copy())

        df.drop('3_phase_watt_total', axis=1, inplace=True)
        
        
        # change col name
        #df.rename(columns={'total_watt_hour': 'mean'}, inplace=True)
        

        all_data.append(df)

In [19]:
# create with all columns to save for later
save_data_col = pd.concat(save_data, ignore_index=True)

In [20]:
# combine all dataframes in list to one dataframe
combined_data = pd.concat(all_data, ignore_index=True)

In [21]:
print(save_data_col.head(10))
print("hi",save_data[1])
print(combined_data.head(10))

              datetime        meter_name  total_watt_hour  3_phase_watt_total
0  2025-07-23 09:40:50  admin_serv_1_mtr          1381508        27995.804688
1  2025-07-23 09:41:54  admin_serv_1_mtr          1381508        36522.058594
2  2025-07-23 09:42:59  admin_serv_1_mtr          1381509        27541.597656
3  2025-07-23 09:44:03  admin_serv_1_mtr          1381510        27613.425781
4  2025-07-23 09:45:08  admin_serv_1_mtr          1381510        27681.099609
5  2025-07-23 09:46:14  admin_serv_1_mtr          1381511        18152.441406
6  2025-07-23 09:47:20  admin_serv_1_mtr          1381511        28280.800781
7  2025-07-23 09:48:26  admin_serv_1_mtr          1381512        26957.160156
8  2025-07-23 09:48:39  admin_serv_1_mtr          1381512        27021.441406
9  2025-07-23 09:49:39  admin_serv_1_mtr          1381512        26907.064453
hi                  datetime        meter_name  total_watt_hour  \
0     2025-07-28 00:00:18  admin_serv_1_mtr          1383795   
1     2025-

In [16]:
save_data_col.to_csv('save_data_col.csv', index=False)

In [17]:
# convert dataframe to csv
combined_data.to_csv('step1.csv', index=False)